# Getting Started With Amazon Lookout for Metrics

Amazon Lookout for Metrics can help you identify anomalies within your data regardless of its origin. By following this notebook you will build out a solution using Poirot to capture incoming data and to detect anomalies within it.

This guide assumes you completed all the work in `README.md`, if you have not, go back to that first then return

## Poirot's Workflow

1. Create a detector and configure its detection properties.
2. Prepare your source data and create an AWS Identity and Access Management (IAM) role that can access the data.
3. Create a dataset:
    1. Provide the location of your source data and the IAM permissions needed to access it. 
    1. Define the metrics that you want to investigate.
    1. Attach the dataset to your detector.
4. Activate the detector.
5. (Optional) Set up alerts to get notified when Poirot detects important outliers.
6. Inspect the detected outliers to figure out their root causes.
7. Provide feedback on the outliers to improve predictor accuracy.

Additionally in the next section after this notebook you will learn how to simulate new data and to stream it to test the service in more detail.

As with nearly all projects involving AWS, the first step is to import the boto3 package then to establish a connection to AWS:

In [1]:
import boto3

In [2]:
session = boto3.Session(region_name = "us-east-1")
poirot = session.client("poirot")

Next validate that you are receiving anything back from this API call below, if you get a 200 code it means you are whitelisted and connecting to the service successfully!

In [3]:
poirot.list_metric_sets()

{'ResponseMetadata': {'RequestId': '37b4e5c3-f9b1-49b7-b798-86ad19dceaa7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 24 Nov 2020 21:37:20 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '4065',
   'connection': 'keep-alive',
   'x-amzn-requestid': '37b4e5c3-f9b1-49b7-b798-86ad19dceaa7',
   'x-amz-apigw-id': 'WiB6kHtYIAMFveQ=',
   'x-amzn-trace-id': 'Root=1-5fbd7d10-2825d6a61b4fc16d6a7d2ea0'},
  'RetryAttempts': 0},
 'MetricSetSummaryList': [{'MetricSetArn': 'arn:aws:poirot:us-east-1:059124553121:MetricSet/DetectorOctober28/october28dataset',
   'AnomalyDetectorArn': 'arn:aws:poirot:us-east-1:059124553121:AnomalyDetector:DetectorOctober28',
   'MetricSetName': 'october28dataset',
   'CreationTime': datetime.datetime(2020, 10, 28, 14, 18, 20, 385000, tzinfo=tzlocal()),
   'LastModificationTime': datetime.datetime(2020, 10, 28, 14, 18, 20, 385000, tzinfo=tzlocal())},
  {'MetricSetArn': 'arn:aws:poirot:us-east-1:059124553121:MetricSet/InitialBackt

### S3 Bucket

As mentioned above, the data needs to exist somehwere, the code below will create a bucket for you to use. 

In [5]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
s3_bucket = account_id + "alfmtestcf"
region = "us-east-1"
s3.create_bucket(Bucket=s3_bucket)

{'ResponseMetadata': {'RequestId': '40229DFA055C88F9',
  'HostId': '+I6bNUuF0+useJ39XaaC174jO+rAjEh2B3Exm0adGXcgLH7EiVD31kKHe2m+4Z7Iiyd2aSNu3EI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '+I6bNUuF0+useJ39XaaC174jO+rAjEh2B3Exm0adGXcgLH7EiVD31kKHe2m+4Z7Iiyd2aSNu3EI=',
   'x-amz-request-id': '40229DFA055C88F9',
   'date': 'Wed, 04 Nov 2020 19:21:07 GMT',
   'location': '/059124553121poirottestcf',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/059124553121poirottestcf'}

## Creating A Detector

Now the basic external resources are ready, so it is time to get started with Amazon Lookout for Metrics (ALFM), that starts with creating a `Detector`.

### Detectors

To detect outliers, ALFM builds a machine learning model that is trained with your source data. This model, called a `detector`, is automatically trained with the machine learning algorithm that best fits your data and use case. You can either provide your historical data for training, if you have any, or get started with real-time data, and ALFM will learn on-the-go. 

You specify the Amazon S3 location that ALFM should continuously monitor for new data, and your detector analyzes your data and returns information about the outliers that it detected. When you create a `detector`, you also specify a `detecting domain` and an `outlier detection frequency`. 

A `detecting domain` helps the detector learn how to best handle a specific business use case, such as advertising and marketing, retail, games, Fintech, software and internet, telecommunications, and more. 

The `outlier detection frequency` specifies how frequently the detector should look for new data, run analysis and alert you with any interesting findings.


Execute the cells below to create a new anomaly detector with a frequency of 5 minutes. 

**NOTE** If you do not have an S3 bucket created for your data, go create one first then come back to these cells.

In [6]:
project = "initial-alfm-testing-cf" # just a string used to name resources such as MetricSet, Detector, etc.

frequency = "PT5M" # one of 'P1D', 'PT1H', 'PT10M' and 'PT5M'

The cell below will leverage the advertising domain, additional domains will be listed on the docs page or within a drop down in the service console.

In [8]:
response = poirot.create_anomaly_detector( 
    AnomalyDetectorName = project + "-detector-1",
    AnomalyDetectorDomain = "ADS",
    AnomalyDetectorDescription = "My Detector",
    AnomalyDetectorConfig = {
        "AnomalyDetectorFrequency" : frequency,
    },
)

anomaly_detector_arn = response["AnomalyDetectorArn"]
anomaly_detector_arn

'arn:aws:poirot:us-east-1:059124553121:AnomalyDetector:initial-poirot-testing-cf-detector-1'

See details of created detector and it's status (should be `INACTIVE` at this point)

In [9]:
poirot.describe_anomaly_detector(AnomalyDetectorArn=anomaly_detector_arn)

{'ResponseMetadata': {'RequestId': '5478715e-f52e-486f-8ea2-82977ed88e68',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 04 Nov 2020 19:21:24 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '465',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5478715e-f52e-486f-8ea2-82977ed88e68',
   'x-amz-apigw-id': 'VfzQSExzoAMFXEA=',
   'x-amzn-trace-id': 'Root=1-5fa2ff34-11df99ce4f7d395706f09d09'},
  'RetryAttempts': 0},
 'AnomalyDetectorArn': 'arn:aws:poirot:us-east-1:059124553121:AnomalyDetector:initial-poirot-testing-cf-detector-1',
 'AnomalyDetectorName': 'initial-poirot-testing-cf-detector-1',
 'AnomalyDetectorDescription': 'My Detector',
 'AnomalyDetectorDomain': 'ADS',
 'AnomalyDetectorConfig': {'AnomalyDetectorFrequency': 'PT5M'},
 'CreationTime': datetime.datetime(2020, 11, 4, 19, 21, 21, 535000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 11, 4, 19, 21, 21, 535000, tzinfo=tzlocal()),
 'Status': 'INACTIVE'}

## Define Metrics

### Measures and Dimensions

`Measures` are variables or key performance indicators on which customers want to detect outliers. Revenue is an example of a measure. `Dimensions` are meta-data that represent categorical information about the measures. 

For example, revenue can have dimensions such as store ID, product category
or city. Customers may want to monitor their data for outliers in revenue for every store ID, product category and city combination, and not just aggregated revenue. You can designate up to five measures and five dimensions per dataset.

### Metrics 


After creating a detector, and mapping your measures and dimensions, ALFM will analyze each combination of these measures and dimensions. For the above example with revenue as the measure and store ID, product category, and city as dimensions, imagine that store ID dimension has 20 unique values (SID1, SID2, etc.), product category has 10 unique values (Gift Card, TV, Phone, Laptop, etc.), and city has 200 unique values (Seattle, San Francisco, Portland, etc.). Each unique combination of measures with the dimension values (e.g. Revenue/SID1/Gift Card/Seattle) is a `metric`. In this case, there are 20 * 10 * 200 = 40,000 metrics. ALFM detects outliers at the most granular level so you are able to pin-point any unexpected behavior in your data.

### Datasets

Measures, dimensions and metrics map to `datasets`, which also contain the Amazon S3 locations of your source data, an IAM role that has both read and write permissions to those Amazon S3 locations, and the rate at which data should be ingested from the source location (the upload frequency and data ingestion delay).


First let's create a role that can work with the ALFM service:


In [ ]:
import poirot_utility as pu
role_name = "ALFMTestRole"
role_arn = pu.get_or_create_iam_role(role_name)

Created ALFMTestRole
Attaching policies
Waiting for a minute to allow IAM role policy attachment to propagate


At this point the choice of MetricType is completely arbitrary as the service does nothing with that info, but the type should be one of legal types per selected domain earlier. Match the metrics (aka measures) and dimensions to what you have in your dataset. In our sample dataset, there is single dimension `product` with values `A` or `B` and only 2 measures: `count` - number of products sold and `price` the sale price (perhaps different vendors have different prices).

In [11]:
s3_path_format = 's3://'+ s3_bucket + '/{{yyyy}}/{{MM}}/{{dd}}/{{HH}}-{{mm}}'

params = {
    "AnomalyDetectorArn": anomaly_detector_arn,
    "MetricSetName" : project + '-metric-set-1',
    "MetricList" : [
        {
            "MetricName" : "count",
            "MetricType" : "CLICKS",
            "AggregationFunction" : "SUM",
        },
        {
            "MetricName" : "price",
            "MetricType" : "VALUE_PER_VISIT",
            "AggregationFunction" : "AVG",
        },
    ],

    "DimensionList" : [ "product" ],

    "TimestampColumn" : {
        "ColumnName" : "timestamp",
        "ColumnFormat" : "yyyy-MM-dd HH:mm:ss",
    },

   #"Delay" : 120, # seconds the detector will wait before attempting to read latest data per current time and detection frequency below
    "MetricSetFrequency" : frequency,

    "MetricSource" : {
        "S3SourceConfig": {
            "RoleArn" : role_arn,
#            "HistoricalDataPathList": [
#                s3_path_training_prefix,
#            ],
            "TemplatedPathList": [
                s3_path_format,
            ],

            "FileFormatDescriptor" : {
                "CsvFormatDescriptor" : {
                    "FileCompression" : "NONE",
                    "Charset" : "UTF-8",
                    "ContainsHeader" : True,
                    "Delimiter" : ",",
#                    "HeaderList" : [
#                        "timestamp",
#                        "product",
#                        "count",
#                        "price"
#                    ],
                    "QuoteSymbol" : '"'
                },
            }
        }
    },
}

response = poirot.create_metric_set( ** params )

metric_set_arn = response["MetricSetArn"]
metric_set_arn

'arn:aws:poirot:us-east-1:059124553121:MetricSet/initial-poirot-testing-cf-detector-1/initial-poirot-testing-cf-metric-set-1'

To see that the metric set was created correctly:

In [12]:
poirot.describe_metric_set(MetricSetArn=metric_set_arn)

{'ResponseMetadata': {'RequestId': '6c147030-1011-402a-895d-d8cdb763d4f9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 04 Nov 2020 19:22:56 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '1289',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6c147030-1011-402a-895d-d8cdb763d4f9',
   'x-amz-apigw-id': 'VfzepHb4IAMF5YA=',
   'x-amzn-trace-id': 'Root=1-5fa2ff90-55da92985462a2c32d98586d'},
  'RetryAttempts': 0},
 'MetricSetArn': 'arn:aws:poirot:us-east-1:059124553121:MetricSet/initial-poirot-testing-cf-detector-1/initial-poirot-testing-cf-metric-set-1',
 'AnomalyDetectorArn': 'arn:aws:poirot:us-east-1:059124553121:AnomalyDetector:initial-poirot-testing-cf-detector-1',
 'MetricSetName': 'initial-poirot-testing-cf-metric-set-1',
 'CreationTime': datetime.datetime(2020, 11, 4, 19, 22, 55, 336000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 11, 4, 19, 22, 55, 336000, tzinfo=tzlocal()),
 'Offset': 0,
 'MetricList': [{'Met

## Activate the Detector

Now that the MetricSet has been specified, we are ready to start training the detector, that's done by activating it.

In [13]:
poirot.activate_anomaly_detector(AnomalyDetectorArn = anomaly_detector_arn)

{'ResponseMetadata': {'RequestId': '8181c107-bbf5-4dd8-82bb-35fdea8ca4a9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 04 Nov 2020 19:22:59 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8181c107-bbf5-4dd8-82bb-35fdea8ca4a9',
   'x-amz-apigw-id': 'VfzfBG8YIAMFXWA=',
   'x-amzn-trace-id': 'Root=1-5fa2ff93-500caf0844fa6b2753b38422'},
  'RetryAttempts': 0}}

## Create Anomaly Alerts:

Once your detector is active, you can attach alerts to it. `Alerts` are customized notifications available via the Amazon Simple Notification Service (SNS), configurable directly via the ALFM console and SDK. These alerts notify you whenever an outlier of a specified severity level is detected. Severity levels are a measure of the urgency or criticality of detected outliers. Alerts are meant to guide you towards relative prioritization of the outliers. ALFM supports Critical, High, Medium, and Low thresholds. For example, you can set an alert on your detector to notify you whenever an outlier with a Medium severity level or greater is detected.


Before we get to creating an alert, 2 additional things are needed:

1. A role giving ALFM access to SNS
2. An SNS topic to deliver the alerts to

The cells below will guide you through creating those and then it is time to create the alert.


In [14]:
import json
import time

In [15]:
iam = boto3.client("iam")
role_name = "ALFMSNSFullAccessCF"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "poirot.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# Now add SNS support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonSNSFullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::059124553121:role/PoirotSNSFullAccessCF


Now create the SNS topic for the alerts:

**UPDATE YOUR CELL NUMBER BELOW!!!** 

In [16]:
sns_client = boto3.client("sns")
topic = sns_client.create_topic(Name="anomalyalertsCF")
topic_arn = topic['TopicArn']

# Change to your cell
number = "+19102842169" # Change to your cell

sns_client.subscribe(
        TopicArn=topic_arn,
        Protocol='sms',
        Endpoint=number  
)

{'SubscriptionArn': 'arn:aws:sns:us-east-1:059124553121:anomalyalertsCF:964ec8e2-63c1-40b6-ad45-3b7472a53ff4',
 'ResponseMetadata': {'RequestId': 'bdface74-4a1f-5ce9-bc7e-966948023754',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bdface74-4a1f-5ce9-bc7e-966948023754',
   'content-type': 'text/xml',
   'content-length': '365',
   'date': 'Wed, 04 Nov 2020 19:24:13 GMT'},
  'RetryAttempts': 0}}

Lastly execute the cell below to configure the alerts to notify your topic.

In [17]:
response = poirot.create_alert(
    Action = {
      "SNSConfiguration": { 
         "RoleArn": role_arn,
         "SnsTopicArn": topic_arn
      }
    },
    AlertDescription = "Test Alert 1",
    AlertName = project + "-alert-1",
    AnomalyDetectorArn = anomaly_detector_arn,
    AlertSensitivityThreshold = 50
)

alert_arn = response["AlertArn"]
alert_arn

'arn:aws:poirot:us-east-1:059124553121:Alert:initial-poirot-testing-cf-alert-1'

To make things easier on yourself we are going to leverage the magic functions of Ipython in order to save a few variables for later. 

Once you have executed the cells below, move on to `2.GenerateDataForALFM.ipynb`

In [18]:
%store project
%store s3_bucket
%store frequency


Stored 'project' (str)
Stored 's3_bucket' (str)
Stored 'frequency' (str)
